In [1]:
import pandas as pd
from pathlib import Path
from tqdm import tqdm
tqdm.pandas()
import re
import numpy as np
import seaborn as sns
import pickle

device='cuda'

In [2]:
def preprocessing_stroke(x):
    text = x.replace('\n',' ')
    text = re.sub('г\.',' ',text)
    text = text.replace('\xa0',' ')
    text = text.replace('\t',' ')
    text = re.sub('[!@#$%^&*_+=\;\]\[\}\{://|«»"№\d]', ' ', text)
    text = re.sub('[\(\)]', ' ', text)
    text = re.sub('-', '', text)
    text = re.sub('[IVN]', ' ', text)
    text = re.sub(' [габвт] ', '', text)
    text = re.sub(' +', ' ', text)
    text = text.strip()
    return text

def prepare_text(x):
    string, text,n = '','',0
    # text = re.sub('п\.','пункте ',text)
    # text = re.sub('ст\.','статья ',text)
    # text = re.sub('ул\.','улица ',text)
    # text = re.sub('т\.ч','том числе ',text)
    texts = []
    for t in x.split('.'):
        string+=' '+t
        text+=' '+t
        if len(string)>100:
            string = preprocessing_stroke(string)
            if len(string)>100:
                # print(text)
                # print('-------------')
                # print(string)
                # print('########################')
                texts.append({'original_text':text,
                              'text':string})
                string=''
                text=''
                n+=1
            
    return texts

In [3]:
#get token and embedding functions

def get_token(text):
    return tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=512,
            return_token_type_ids=False,
            truncation=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt')

def get_embeddings(text):
    encoding = get_token(text)
    out = {
      'text': text,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten()
      }

    input_ids = out["input_ids"].to(device)
    attention_mask = out["attention_mask"].to(device)

    outputs = model(
        input_ids=input_ids.unsqueeze(0),
        attention_mask=attention_mask.unsqueeze(0)
    )
    return outputs[1].cpu().detach().numpy()[0]

In [4]:
import torch
from transformers import BertTokenizer

model = torch.load('bert-base.pt', map_location=torch.device('cpu'))
model = model.bert
pass

#load tokenizer
tokenizer = BertTokenizer.from_pretrained('sberbank-ai/ruBert-base')

with open('svm.pickle', 'rb') as handle:
    svm = pickle.load(handle)

/home/ubuntu/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ubuntu/.local/lib/python3.8/site-packages/sklearn/base.py:288: UserWarning: Trying to unpickle estimator StandardScaler from version 1.1.1 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/ubuntu/.local/lib/python3.8/site-packages/sklearn/base.py:288: UserWarning: Trying to unpickle estimator SVC from version 1.1.1 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  w

In [6]:
def get_token(text):
    return tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=512,
            return_token_type_ids=False,
            truncation=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt')

def get_embeddings(text):
    encoding = get_token(text)
    out = {
      'text': text,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten()
      }

    input_ids = out["input_ids"].to(device)
    attention_mask = out["attention_mask"].to(device)

    outputs = model(
        input_ids=input_ids.unsqueeze(0),
        attention_mask=attention_mask.unsqueeze(0)
    )
    return outputs[1].cpu().detach().numpy()[0]

In [ ]:
from flask import Flask,request
from flask import jsonify


app = Flask(__name__)
app.secret_key = '1khagd9OKIUdjUGblvHc2mCIGKE_4K3QYZks3Nm8zmWbcj4pn'


@app.route('/api', methods=['POST'])
def register():
        text = request.json
        data = prepare_text(text['text'])
        result = []
        print(data)
        for d in data:
            result.append(get_embeddings(data['text']))
        
        result = np.array(result)
        print(result)
        results = np.argmax(svm.predict(result))
        
        for i,d in enumerate(data):
            d['text_type']=results[i]
        
        answer = {'answer':[d for d in data]}
            
        
        return jsonify(answer)

if __name__ == "__main__":
    app.run(host='localhost', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:5000
Press CTRL+C to quit
[2022-12-16 15:45:14,356] ERROR in app: Exception on /api [POST]
Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.8/site-packages/flask/app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/ubuntu/.local/lib/python3.8/site-packages/flask/app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/ubuntu/.local/lib/python3.8/site-packages/flask/app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/ubuntu/.local/lib/python3.8/site-packages/flask/app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/tmp/ipykernel_5236/4082689647.py", line 17, in register
    result.append(get_embeddings(data['text']))
TypeError: list indices must be integers or slices, not str
127.0.0.1 - - [16/Dec/2022 15:45:14] "POST /api HTTP/1.1" 500 -


{'text': '\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nДОГОВОР № \nкупли-продажи\n\nг. Москва                                                                                                                          «  »                     201  г.\n\nОбщество с ограниченной ответственностью «Форгрейт», именуемое в дальнейшем «ПРОДАВЕЦ», в\nлице Заместителя генерального директора Суровой Инги Александровны, действующей на основании\nДоверенности № 001 от 14.01.2016 г., с одной стороны, и                                                                        ,\nименуемое в дальнейшем «ПОКУПАТЕЛЬ», в лице                                                                         ,\nдействующего на основании               , с другой стороны, вместе именуемые «СТОРОНЫ», заключили\nнастоящий Договор о нижеследующем:\n\n1. Предмет Договора\n\n1.1. Предметом настоящего Договора является купля - продажа мороженной рыбы, рыбопродукции и\nморепродуктов (далее - “ТО

In [ ]:
's'

In [ ]:
import os
import flask_ngrok
path = os.path.abspath(flask_ngrok.__file__)
path